# Market Data Integration Guide

## Overview
This guide covers integrating with TastyTrade's market data API using Python WebSocket implementation, with a focus on asyncio-based approach.

## TastyTrade API Resources
- [Developer Portal](https://developer.tastytrade.com/)
- [Streaming Market Data Documentation](https://developer.tastytrade.com/streaming-market-data/)
- [DXLink Protocol Reference](https://demo.dxfeed.com/dxlink-ws/debug/#/protocol)

## WebSocket Implementation Details

### Selected Library
We'll be using the [websockets](https://websockets.readthedocs.io/en/stable/) library for Python, which provides high-level WebSocket implementation (not to be confused with the lower-level `websocket` library).

### Key Resources
- **Documentation**: Complete guides available on [readthedocs.io](https://websockets.readthedocs.io/en/stable/)
- **Reference Implementation**: [Example using Threading](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) by [LordKaT](https://github.com/LordKaT)

### Technical Approach
The implementation will use `asyncio` due to its comprehensive [feature support](https://websockets.readthedocs.io/en/stable/reference/features.html) in the websockets library.

### AsyncIO Learning Resources
For better understanding of the asyncio implementation:
- [How does async/await work in Python 3.5](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Cooperative multitasking with Coroutines](https://pymotw.com/3/asyncio/coroutines.html)
- [A Curious course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)

## Implementation Notes
While previous experience has been primarily with traditional threading, this project will explore asyncio-based implementation for its potential benefits in handling WebSocket connections.

In [1]:
import logging
import asyncio
from tastytrade.utilties import setup_logging
from tastytrade import Credentials
from tastytrade.session import AsyncSessionHandler, WebSocketManager, DXLinkClient

logging.getLogger().handlers.clear()
logger = logging.getLogger(__name__)
setup_logging(logging.DEBUG)

TEST = True
ENV = "Live"

In [2]:
channel = 1

session = await AsyncSessionHandler.create(Credentials(env=ENV))

async with WebSocketManager(session) as websocket:
    dxlink_client = DXLinkClient(websocket)

    await dxlink_client.request_channel(channel)
    await dxlink_client.setup_feed(channel)
    await dxlink_client.subscribe_to_feed(channel)
    await asyncio.sleep(1)

await session.close()

2024-11-23 21:36:33 - INFO:tastytrade.session:226:Session created successfully
2024-11-23 21:36:33 - DEBUG:tastytrade.session:237:Retrieved dxlink token
2024-11-23 21:36:33 - DEBUG:websockets.client:175:= connection is CONNECTING
2024-11-23 21:36:34 - DEBUG:websockets.client:306:> GET /realtime HTTP/1.1
2024-11-23 21:36:34 - DEBUG:websockets.client:308:> Host: tasty-openapi-ws.dxfeed.com
2024-11-23 21:36:34 - DEBUG:websockets.client:308:> Upgrade: websocket
2024-11-23 21:36:34 - DEBUG:websockets.client:308:> Connection: Upgrade
2024-11-23 21:36:34 - DEBUG:websockets.client:308:> Sec-WebSocket-Key: qAa6XgXROXJk1wVN8oOX3w==
2024-11-23 21:36:34 - DEBUG:websockets.client:308:> Sec-WebSocket-Version: 13
2024-11-23 21:36:34 - DEBUG:websockets.client:308:> Sec-WebSocket-Extensions: permessage-deflate; client_max_window_bits
2024-11-23 21:36:34 - DEBUG:websockets.client:308:> User-Agent: Python/3.11 websockets/14.1
2024-11-23 21:36:34 - DEBUG:websockets.client:329:< HTTP/1.1 101 Switching Prot

In [3]:
exlink = WebSocketManager(session)
txlink = WebSocketManager(session)
print(id(exlink) == id(txlink))

True


In [4]:
txlink.__dict__

{'session': <tastytrade.session.AsyncSessionHandler at 0x1261bf850>,
 'url': 'wss://tasty-openapi-ws.dxfeed.com/realtime',
 'token': 'dGFzdHksYXBpLCwxNzMyNDYyMjU4LDE3MzIzNzU4NTgsVWZmODBiM2M3LTJiZjgtNDRkYi1hZmZkLWZiOWFhOGU5NDZkNQ.LhG8v6UhUxvn0VhXSEousB6wqDl9Bx1m-gYQZJA-TOI',
 'message_handler': <tastytrade.messages.MessageHandler at 0x126203850>,
 'channels': {},
 'lock': <asyncio.locks.Lock object at 0x106033dd0 [unlocked]>}

In [5]:
exlink.sessions

{<tastytrade.session.AsyncSessionHandler at 0x1261bf850>: <tastytrade.session.WebSocketManager at 0x105887d50>}

In [6]:
await exlink.open()

2024-11-23 21:36:35 - DEBUG:websockets.client:175:= connection is CONNECTING
2024-11-23 21:36:35 - DEBUG:websockets.client:306:> GET /realtime HTTP/1.1
2024-11-23 21:36:35 - DEBUG:websockets.client:308:> Host: tasty-openapi-ws.dxfeed.com
2024-11-23 21:36:35 - DEBUG:websockets.client:308:> Upgrade: websocket
2024-11-23 21:36:35 - DEBUG:websockets.client:308:> Connection: Upgrade
2024-11-23 21:36:35 - DEBUG:websockets.client:308:> Sec-WebSocket-Key: Fp+NKggtmD+U6t6k5J8itg==
2024-11-23 21:36:35 - DEBUG:websockets.client:308:> Sec-WebSocket-Version: 13
2024-11-23 21:36:35 - DEBUG:websockets.client:308:> Sec-WebSocket-Extensions: permessage-deflate; client_max_window_bits
2024-11-23 21:36:35 - DEBUG:websockets.client:308:> User-Agent: Python/3.11 websockets/14.1
2024-11-23 21:36:35 - DEBUG:websockets.client:329:< HTTP/1.1 101 Switching Protocols
2024-11-23 21:36:35 - DEBUG:websockets.client:331:< Connection: upgrade
2024-11-23 21:36:35 - DEBUG:websockets.client:331:< Date: Sun, 24 Nov 2024 

In [7]:
await asyncio.sleep(5)
await exlink.close()

2024-11-23 21:36:35 - DEBUG:websockets.client:599:< TEXT '{"type":"SETUP","channel":0,"keepaliveTimeout":...1.2.3-20240912-171526"}' [116 bytes]
2024-11-23 21:36:35 - DEBUG:websockets.client:599:< TEXT '{"type":"AUTH_STATE","channel":0,"state":"UNAUTHORIZED"}' [56 bytes]
2024-11-23 21:36:35 - INFO:tastytrade.messages:53:SETUP
2024-11-23 21:36:35 - INFO:tastytrade.messages:58:AUTH_STATE:UNAUTHORIZED
2024-11-23 21:36:35 - DEBUG:websockets.client:599:< TEXT '{"type":"AUTH_STATE","channel":0,"state":"AUTHO...7ae-8957-e356b39b225d"}' [102 bytes]
2024-11-23 21:36:35 - INFO:tastytrade.messages:58:AUTH_STATE:AUTHORIZED
2024-11-23 21:36:40 - INFO:tastytrade.session:341:Websocket listener stopped
2024-11-23 21:36:40 - DEBUG:websockets.client:599:< CLOSE 1000 (OK) [2 bytes]
2024-11-23 21:36:40 - DEBUG:websockets.client:649:< EOF
2024-11-23 21:36:40 - DEBUG:websockets.client:757:> EOF
2024-11-23 21:36:40 - DEBUG:websockets.client:175:= connection is CLOSED
2024-11-23 21:36:40 - DEBUG:websockets.cl

2024-11-23 21:36:40 - DEBUG:websockets.client:745:> CLOSE 1000 (OK) [2 bytes]
2024-11-23 21:36:40 - DEBUG:websockets.client:175:= connection is CLOSING
2024-11-23 21:36:40 - INFO:tastytrade.session:316:Websocket closed


In [8]:
exlink.sessions

{}

In [9]:
# Injector Module Example

# def configure(binder):
#     binder.bind(Credentials, to=Credentials(env=ENV))


# injector = Injector([configure])
# session = injector.get(SessionHandler)

# session.get_api_quote_token()

# session = SessionHandler.create(Credentials(env="Test"))

# if TEST:
#     session.create_session(Credentials(env=ENV))

In [10]:
# new_response = request_options_chains(session, "SPXW")
# options_chains_df = pd.DataFrame(new_response.json()["data"]["items"])

In [11]:
# columns = [
#     "underlying-symbol",
#     "option-type",
#     "strike-price",
#     "streamer-symbol",
#     "symbol",
# ]

# rows = (options_chains_df["days-to-expiration"] == 0) & (
#     options_chains_df["strike-price"].str.contains(r"588|590|591", regex=True)
# )

# options_chains_df.loc[rows, columns]

In [12]:
# session.close_session()